In [1]:
from glob import glob
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import json
import random

In [2]:
csv_t2020_df = pd.read_csv('/home/llm-hackathon/Downloads/fixed-data-csv/all-reports-available-xrays-through2020-final.csv')
csv_a2020_df = pd.read_csv('/home/llm-hackathon/Downloads/fixed-data-csv/all-reports-available-xrays-after2020-final.csv')
csv_ids_through_2020 = csv_t2020_df['AccessionId']
csv_ids_after2020 = csv_a2020_df['AccessionId']
csv_t2020_df_indexed = csv_t2020_df.set_index('AccessionId')

In [3]:
img_fp = '/data/UCSD_cxr/jpg/'
img_ids = glob('*.jpg', root_dir=img_fp)
img_ids.sort()
img_acc_ids = [int(re.search(r'[0-9]+',img_id)[0]) for img_id in img_ids]

In [4]:
print('# Img Acc Ids, # ids thru 2020, # ids after 2020, # img in thru 2020, # img in after 2020')
len(img_acc_ids), len(csv_ids_through_2020), len(csv_ids_after2020), len(set(csv_ids_through_2020).intersection(set(img_acc_ids))), len(set(csv_ids_after2020).intersection(set(img_acc_ids))), 

# Img Acc Ids, # ids thru 2020, # ids after 2020, # img in thru 2020, # img in after 2020


(23711, 59965, 66495, 23707, 25)

In [37]:
missing_ids = list(set(img_acc_ids) - set(csv_ids_through_2020).intersection(set(img_acc_ids)))
missing_ids

[51579818, 50647186, 52811952, 50556838]

In [38]:
# shuffle ids
random.seed(10)
random.shuffle(img_ids)

In [39]:
split_idx = len(img_ids) - len(img_ids)//10
split_idx

21340

In [43]:
train_img_ids = img_ids[:split_idx]
test_img_ids = img_ids[split_idx:]

In [44]:
train_img_acc_ids = [int(re.search(r'[0-9]+',img_id)[0]) for img_id in train_img_ids]
test_img_acc_ids = [int(re.search(r'[0-9]+',img_id)[0]) for img_id in test_img_ids]

In [45]:
def generate_json(img_ids, save_json_fp, data_fp, question):
    json_output = []

    for img_id in tqdm(img_ids):
        img_acc_id = int(re.search(r'[0-9]+', img_id)[0])
        patient_id = img_id.split('.')[0]
        if img_acc_id in csv_t2020_df_indexed.index:
            prompt = csv_t2020_df_indexed.loc[img_acc_id, 'prompt'] + question
            answer = csv_t2020_df_indexed.loc[img_acc_id, 'answer']

            patient_data = {
                "id": patient_id,
                "image": data_fp + img_id,
                "conversations": [
                    {
                        "from": "human",
                        "value": f"{prompt}"
                    },
                    {
                        "from": "gpt",
                        "value": f"{answer}"

                    }

                ]
            }

            json_output.append(patient_data)

    with open(save_json_fp, 'w') as f:
        json.dump(json_output, f, indent=4)

    print('Dumped JSON')

In [29]:
# Generate train json
# save_json_fp = '/home/llm-hackathon/LLaVA/data/train_patient_finding_impression.json' 
# data_fp = '/home/llm-hackathon/LLaVA/data/jpg/'
# question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'
# generate_json(train_img_ids, save_json_fp, data_fp, question)

100%|██████████| 21340/21340 [00:00<00:00, 94685.76it/s]


Dumped JSON


In [30]:
# Generate test json 
# save_json_fp = '/home/llm-hackathon/LLaVA/data/test_patient_finding_impression.json' 
# data_fp = '/home/llm-hackathon/LLaVA/data/jpg/'
# question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'
# generate_json(test_img_ids, save_json_fp, data_fp, question)

100%|██████████| 2371/2371 [00:00<00:00, 75730.44it/s]

Dumped JSON


In [1]:
# Generate initial json
data_fp = img_fp
json_output = []
save_json_fp = '/home/llm-hackathon/LLaVA/data/patient_finding_impression.json'
question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'

for img_id in tqdm(img_ids):
    img_acc_id = int(re.search(r'[0-9]+', img_id)[0])
    patient_id = img_id.split('.')[0]
    if img_acc_id in csv_t2020_df_indexed.index:
        prompt = csv_t2020_df_indexed.loc[img_acc_id, 'prompt'] + question
        answer = csv_t2020_df_indexed.loc[img_acc_id, 'answer']

        patient_data = {
            "id": patient_id,
            "image": data_fp + img_id,
            "conversations": [
                {
                    "from": "human",
                    "value": f"{prompt}"
                },
                {
                    "from": "gpt",
                    "value": f"{answer}"

                }

            ]
        }

        json_output.append(patient_data)

with open(save_json_fp, 'w') as f:
    json.dump(json_output, f, indent=4)

print('Dumped JSON')

NameError: name 'tqdm' is not defined

In [7]:
# Generate questions file.
data_fp = img_fp
save_json_fp = '/home/llm-hackathon/LLaVA/data/questions_finding_impression.jsonl'
question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?\nAlso, identify from the following list of conditions--[Edema, Pneumonia, Pneumothorax, Atelectasis, Lung Lesion, Consolidation, Cardiomegaly, Pleural Effusion, Enlarged Cardiom, Lung Opacity]--and tell me which one(s) this X-ray shows.'
with open(save_json_fp, 'w') as f:

    for img_id in tqdm(img_ids):
        img_acc_id = int(re.search(r'[0-9]+', img_id)[0])
        patient_id = img_id.split('.')[0]
        if img_acc_id in csv_t2020_df_indexed.index:
            prompt = csv_t2020_df_indexed.loc[img_acc_id, 'prompt'] + question
            patient_data = {
                "question_id": patient_id,
                "image": data_fp + img_id,
                "text": f"{prompt}",
                "category": "",
            }
            f.write(json.dumps(patient_data) + "\n")


print('Dumped JSON')

100%|██████████| 23711/23711 [00:00<00:00, 108745.99it/s]

Dumped JSON


In [3]:
answers = []
with open("/home/llm-hackathon/LLaVA/data/answers_finding_impression.jsonl", 'r') as f:
    for line in f.readlines():
        output = json.loads(line)
        answers.append(output)

In [46]:
test = pd.read_json('/home/llm-hackathon/LLaVA/data/test_patient_finding_impression.json')

In [47]:
def break_convos(df, col1, col2):
    '''takes in conversations column and breaks it up into separate columns for data frame'''
    ser = df['conversations']
    prompt = pd.json_normalize(ser.apply(lambda x: x[0]))['value']
    impressions = pd.json_normalize(ser.apply(lambda x: x[1]))['value']

    df[col1] = prompt
    df[col2] = impressions

    df = df.drop(axis=1, columns=['conversations'])

    return df

In [48]:
test.head()

,id,image,conversations
0,Najitir_52925711,/home/llm-hackathon/LLaVA/data/jpg/Najitir_529...,"[{'from': 'human', 'value': 'AUTHOR: Liau, Joy..."
1,Lastrolu_52227435,/home/llm-hackathon/LLaVA/data/jpg/Lastrolu_52...,"[{'from': 'human', 'value': 'AUTHOR: Jacobs, K..."
2,Kuluegan_52668517,/home/llm-hackathon/LLaVA/data/jpg/Kuluegan_52...,"[{'from': 'human', 'value': 'AUTHOR: Hahn, Lew..."
3,Phinesdreek_50394671,/home/llm-hackathon/LLaVA/data/jpg/Phinesdreek...,"[{'from': 'human', 'value': 'AUTHOR: Hsiao, Al..."
4,Nenieri_52789390,/home/llm-hackathon/LLaVA/data/jpg/Nenieri_527...,"[{'from': 'human', 'value': 'AUTHOR: Hahn, Lew..."


In [49]:
break_convos(test, 'col1', 'col2')

,id,image,col1,col2
0,Najitir_52925711,/home/llm-hackathon/LLaVA/data/jpg/Najitir_529...,"AUTHOR: Liau, Joy \nCLINICAL HISTORY: Chest pa...",FINDINGS:\nLow lung volumes with bibasal atele...
1,Lastrolu_52227435,/home/llm-hackathon/LLaVA/data/jpg/Lastrolu_52...,"AUTHOR: Jacobs, Kathleen \nCLINICAL HISTORY: B...",FINDINGS:\nModerate lung expansion. Septal thi...
2,Kuluegan_52668517,/home/llm-hackathon/LLaVA/data/jpg/Kuluegan_52...,"AUTHOR: Hahn, Lewis \nCLINICAL HISTORY: Wheezi...",FINDINGS:\nLines and Tubes: None\nMediastinum:...
3,Phinesdreek_50394671,/home/llm-hackathon/LLaVA/data/jpg/Phinesdreek...,"AUTHOR: Hsiao, Albert \nCLINICAL HISTORY: Shor...",FINDINGS:\nStable enlarged cardiomediastinal s...
4,Nenieri_52789390,/home/llm-hackathon/LLaVA/data/jpg/Nenieri_527...,"AUTHOR: Hahn, Lewis \nCLINICAL HISTORY: Chest ...",FINDINGS:\nLines and Tubes: None\nMediastinum:...
...,...,...,...,...
2365,Udusay_50669708,/home/llm-hackathon/LLaVA/data/jpg/Udusay_5066...,"AUTHOR: Ladd, William \nCLINICAL HISTORY: Ches...",FINDINGS:\nSee impression.\n\nIMPRESSION: Norm...
2366,Jojoli_52118594,/home/llm-hackathon/LLaVA/data/jpg/Jojoli_5211...,"AUTHOR: Weihe, Elizabeth \nCLINICAL HISTORY: S...",FINDINGS:\nMediastinum: Enlarged cardiac silho...
2367,Fequera_51962963,/home/llm-hackathon/LLaVA/data/jpg/Fequera_519...,"AUTHOR: Friend, Christopher \nCLINICAL HISTORY...",FINDINGS:\nHeart size normal. Pulmonary vascul...
2368,Edaney_50990038,/home/llm-hackathon/LLaVA/data/jpg/Edaney_5099...,"AUTHOR: Weihe, Elizabeth \nCLINICAL HISTORY: C...",FINDINGS: \nSingle portable view of the che...


In [50]:
test = test.drop(axis=1, columns=['col2', 'conversations'])

In [51]:
test = test.iloc[1501:]
test

,id,image,col1
1501,Joshuku_52000948,/home/llm-hackathon/LLaVA/data/jpg/Joshuku_520...,"AUTHOR: Horowitz, Michael \nCLINICAL HISTORY: ..."
1502,Nefafog_52713974,/home/llm-hackathon/LLaVA/data/jpg/Nefafog_527...,"AUTHOR: Kligerman, Seth \nCLINICAL HISTORY: Ch..."
1503,Quadupen_51674058,/home/llm-hackathon/LLaVA/data/jpg/Quadupen_51...,"AUTHOR: Yen, Andrew \nCLINICAL HISTORY: CHF\nB..."
1504,Coupadbreb_51056928,/home/llm-hackathon/LLaVA/data/jpg/Coupadbreb_...,"AUTHOR: Jacobs, Kathleen \nCLINICAL HISTORY: S..."
1505,Putomo_51739064,/home/llm-hackathon/LLaVA/data/jpg/Putomo_5173...,"AUTHOR: Jacobs, Kathleen \nCLINICAL HISTORY: H..."
...,...,...,...
2365,Udusay_50669708,/home/llm-hackathon/LLaVA/data/jpg/Udusay_5066...,"AUTHOR: Ladd, William \nCLINICAL HISTORY: Ches..."
2366,Jojoli_52118594,/home/llm-hackathon/LLaVA/data/jpg/Jojoli_5211...,"AUTHOR: Weihe, Elizabeth \nCLINICAL HISTORY: S..."
2367,Fequera_51962963,/home/llm-hackathon/LLaVA/data/jpg/Fequera_519...,"AUTHOR: Friend, Christopher \nCLINICAL HISTORY..."
2368,Edaney_50990038,/home/llm-hackathon/LLaVA/data/jpg/Edaney_5099...,"AUTHOR: Weihe, Elizabeth \nCLINICAL HISTORY: C..."


In [59]:
(img_fp + '/' + (test.iloc[0]['image']).split('/')[-1])

'/data/UCSD_cxr/jpg/Joshuku_52000948.jpg'

In [61]:
#test['image'] = test['image'].apply(lambda x: img_fp + '/' + (x.split('/'))[-1])1

In [74]:
data_fp = img_fp + '/'
save_json_fp = '/home/llm-hackathon/LLaVA/data/partial_test_questions_finding_impression.jsonl'#'/home/llm-hackathon/LLaVA/data/test_questions_finding_impression.jsonl'
question = '\nBased on AUTHOR and CLINICAL HISTORY, suppose you were a radiologist on X-RAY,  could you provide a detailed report from this chest X-ray?'
with open(save_json_fp, 'w') as f:

    for img_id in tqdm(test['id']):
        img_acc_id = int(re.search(r'[0-9]+', img_id)[0])
        patient_id = img_id.split('.')[0]
        if img_acc_id in csv_t2020_df_indexed.index:
            prompt = csv_t2020_df_indexed.loc[img_acc_id, 'prompt'] + question
            patient_data = {
                "question_id": patient_id,
                "image": data_fp + img_id + '.jpg',
                "text": f"{prompt}",
                "category": "",
            }
            f.write(json.dumps(patient_data) + "\n")


print('Dumped JSON')

100%|██████████| 869/869 [00:00<00:00, 68313.19it/s]

Dumped JSON


In [65]:
len(set(test['id']).difference(set(img_ids)))

869